In [ ]:
!git clone https://github.com/RajamannarAanjaram/EVA-6.git

In [ ]:
!pip install -U albumentations
!pip install -U torchsummary
!pip install -U grad-cam
!pip install -U ttach

In [ ]:
import sys
sys.path.append("./EVA-6/") 

In [ ]:
from utils.DataLoader import Loader,mean_std,DeNorm
from src.sutom_resnet import *
from src.optimise import *
from src.plots import Plots
import matplotlib.pyplot as plt
from utils.helper import get_mean_std


from pytorch_grad_cam import GradCAM, XGradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image


# import seaborn as sns
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [ ]:
loader = Loader(batch_size=256)
train_transform,test_transform = loader.transform()
train_data,test_data = loader.Loader(train_transform,test_transform, cuda=False)

## Model Creation & Training

In [ ]:
model, summary = ModelLoader.modelsummary(inputsize=(3,32,32))

In [ ]:
use_cuda = torch.cuda.is_available()
device = 'cuda:0' if use_cuda else 'cpu'

In [ ]:
(train_acc, train_losses, test_acc, test_losses), modelfinal = learner(model,train_data,test_data,
                                                                       epochs=40,optimiser= 'Adam',
                                                                       scheduler=False,device = device)

## Loss Curves

In [ ]:
Plots.stat_graph(train_acc, train_losses, test_acc, test_losses)

## Misclassifications

In [ ]:
Plots.miscImages(modelfinal,test_data,device)
plt.show()

## Class Level Accuracy

In [ ]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
    for data in test_data:
        images, labels = data
        images = images["image"].to(device)
        labels = labels.to(device)
        outputs = modelfinal(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

## CAM Visualization

In [ ]:
# using layer 3 as it's 8x8 in width & height
target_layer = model.layer3[-1]

# Construct the CAM object once, and then re-use it on many images:
cam =  GradCAM(model=model, target_layer=target_layer, use_cuda=device)
denorm = DeNorm()

In [ ]:
for idx in range(0,10) :
    Plots.plot_grad_cam(cam,images[idx:idx+1],None,denorm)
    